In [23]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import pandas as pd
iris = load_iris()
iris.target_names
dir(iris)
iris.feature_names
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['Flower'] = iris.target
df['Flower'] = df['Flower'].apply(lambda x:iris.target_names[x])
df[47:52]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [35]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=.2)
model = SVC()
model.fit(iris.data,iris.target)
model.score(x_test,y_test)#score been to be chaning whenever it is implemented
model.predict(x_test)

C:\Users\91894\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([1, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1,
       0, 2, 2, 0, 0, 0, 1, 1])

In [56]:
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(kernel = 'rbf',C=10, gamma = 'auto'),iris.data,iris.target,cv=5)
cross_val_score(SVC(kernel = 'linear',C=20, gamma = 'auto'),iris.data,iris.target,cv=5)
cross_val_score(SVC(kernel = 'rbf',C=30, gamma = 'auto'),iris.data,iris.target,cv=5)
#the above mentioned code in for loop:
import numpy as np
kernel = ['rbf','linear']
C = [10,20,30]
avg_scr = {}
for kval in kernel:
    for cval in C:
        cv_score = cross_val_score(SVC(kernel = kval, C = cval, gamma = 'auto'),iris.data,iris.target,cv=5)
        avg_scr[kval + '-' + str(cval)] =np.average(cv_score)
        
avg_scr

{'rbf-10': 0.9800000000000001,
 'rbf-20': 0.9666666666666668,
 'rbf-30': 0.96,
 'linear-10': 0.9733333333333334,
 'linear-20': 0.9666666666666666,
 'linear-30': 0.96}

In [87]:
#solution for heavy for loop if the parameters are large
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(SVC(gamma = 'auto'),{'C':[1,10,20],'kernel':['rbf','linear']},cv=5,return_train_score=True)
gs.fit(iris.data,iris.target)
gs.cv_results_#messy to view the mean_test_score
df = pd.DataFrame(gs.cv_results_)#converting the gs.cv_results into dataframe
df[['param_C','param_kernel','mean_test_score']]#easy form of viewing requied info
dir(gs)
gs.best_params_
gs.best_score_
gs._estimator_type


'classifier'

In [97]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma = 'auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
    },
    cv=5,
    return_train_score=False,
    n_iter = 2)
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)
df[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [110]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
model_params = {'support_vector_machine':{'model':SVC(gamma = 'auto'),
                      'params':{'C':[1,10,22],
                                'kernel':['rbf','linear']
                               }
                      },
                
                'random_forest':{'model':RandomForestClassifier(),
                                 'params':{'n_estimators':[1,5,10]}
                                },
                
                'logistic_regression':{'model':LogisticRegression(solver='liblinear',multi_class='auto'),
                                       'params':{'C':[1,5,10]}
                                      }
               }
score=[]
for model_name,mp in model_params.items():
    gs=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    gs.fit(iris.data,iris.target)
    score.append({'model':model_name,
                 'best_score':gs.best_score_,
                 'best_params':gs.best_params_})
df=pd.DataFrame(score,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,support_vector_machine,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
